In [41]:
import pandas as pd

In [42]:
csv_path = "resource/airport (1).csv"

In [43]:
airport = pd.read_csv(csv_path, encoding="utf-8")
airport.head()

,airport_name,link,title,author,author_country,date,content,experience_airport,date_visit,type_traveller,overall_rating,queuing_rating,terminal_cleanliness_rating,terminal_seating_rating,terminal_signs_rating,food_beverages_rating,airport_shopping_rating,wifi_connectivity_rating,airport_staff_rating,recommended
0,aalborg-airport,/airport-reviews/aalborg-airport,Aalborg Airport customer review,Klaus Malling,Denmark,2014-02-11,A small very effective airport with few flight...,NaN,NaN,NaN,9.0,5.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,1
1,aalborg-airport,/airport-reviews/aalborg-airport,Aalborg Airport customer review,S Kroes,Netherlands,2013-02-13,This is a nice and modern airport at the momen...,NaN,NaN,NaN,9.0,5.0,4.0,NaN,NaN,NaN,4.0,NaN,NaN,1
2,aalborg-airport,/airport-reviews/aalborg-airport,Aalborg Airport customer review,M Andersen,Denmark,2012-08-07,A very nice airy terminal - that seems modern ...,NaN,NaN,NaN,9.0,5.0,5.0,NaN,NaN,NaN,4.0,NaN,NaN,1
3,aalborg-airport,/airport-reviews/aalborg-airport,Aalborg Airport customer review,Paul Van Alsten,France,2011-05-22,AMS-AAL and quite satisfied with this regional...,NaN,NaN,NaN,5.0,5.0,5.0,NaN,NaN,NaN,3.0,NaN,NaN,0
4,aalborg-airport,/airport-reviews/aalborg-airport,Aalborg Airport customer review,K Fischer,NaN,2010-08-04,Very quick check-inn and security screening. N...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [44]:
airport.reset_index(inplace=True)
airport['index'] = airport.index
airport = airport[["airport_name","overall_rating"]]

airport

,airport_name,overall_rating
0,aalborg-airport,9.0
1,aalborg-airport,9.0
2,aalborg-airport,9.0
3,aalborg-airport,5.0
4,aalborg-airport,4.0
...,...,...
17716,zurich-airport,3.0
17717,zurich-airport,4.0
17718,zurich-airport,3.0
17719,zurich-airport,5.0


In [53]:
airport_rating =airport[airport.overall_rating>9]
airport_rating


,airport_name,overall_rating
83,abu-dhabi-airport,10.0
149,abu-dhabi-airport,10.0
258,adelaide-airport,10.0
259,adelaide-airport,10.0
268,adelaide-airport,10.0
...,...,...
17671,zurich-airport,10.0
17673,zurich-airport,10.0
17674,zurich-airport,10.0
17675,zurich-airport,10.0


In [56]:

airport_rating=airport_rating.drop_duplicates(subset=airport_rating[["airport_name","overall_rating"]])


In [57]:
airport_rating

,airport_name,overall_rating
83,abu-dhabi-airport,10.0
258,adelaide-airport,10.0
311,ahmedabad-airport,10.0
379,alice-springs-airport,10.0
405,amman-queen-alia-airport,10.0
...,...,...
17497,wichita-airport,10.0
17506,winnipeg-airport,10.0
17516,wroclaw-strachowice-airport,10.0
17548,yangon-airport,10.0
